<a href="https://colab.research.google.com/github/poojamahajan0712/COVID_tweet_analysis/blob/master/Covid_Tweet_Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
#!pip install contractions
#import nltk
#nltk.download('stopwords')

import pandas as pd
import seaborn as sns
import contractions
import matplotlib.pyplot as plt

from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

pd.options.display.max_colwidth = 600
pd.options.display.max_columns=500

In [6]:
# was getting error UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc2 in position 254: unexpected end of data 
#so used encoding='latin1' 

df = pd.read_csv('Corona_NLP_train.csv',encoding='latin1')
df.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order,Positive
2,3801,48753,Vagabonds,16-03-2020,"Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P",Positive
3,3802,48754,NaN,16-03-2020,"My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\r\r\n#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j",Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don't panic. It causes shortage...\r\r\n\r\r\n#CoronavirusFrance #restezchezvous #StayAtHome #confinement https://t.co/usmuaLq72n",Extremely Negative


In [7]:
# shape of the data
print('Shape of the data',df.shape)

## Text Cleaning  and steps -- some basic steps can be done 
df['OriginalTweet'].head()

# on checking tweets - we can see lot of special character tags , urls, contractions

Shape of the data (41157, 6)


0                                                                                                                                                                                                                       @MeNyrbie @Phil_Gahan @Chrisitv https://t.co/iFz9FAn2Pa and https://t.co/xX6ghGFzCC and https://t.co/I2NlzdxNo8
1                                                                                         advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order
2                                                                                                                                                                                                   Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P
3     My food st

In [8]:
# creating a column as number of mentions
df['cnt_mentions']=df['OriginalTweet'].apply(lambda x :len([t for t in x.split() if t.startswith('@') ]))
df['cnt_mentions'].describe()

count    41157.000000
mean         0.416916
std          1.116992
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max         21.000000
Name: cnt_mentions, dtype: float64

In [9]:
## so we saw max value for mentions is 21 , let's check how these tweets look like -- seems like all are just tags and thus can be removed.
df[df['cnt_mentions']==21]



,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,cnt_mentions
22994,26793,71745,NaN,25-03-2020,@twigliot @DreaVilleneuve @tedbauer2003 @TheComicHunter @biggreenpen @MrLexJay @JimCanto @ideabloke @JoelRRenner @GabrielSurfCat @BrendaMKramer @YEPBusiness @katyb_spencer @joshmccormack @Renoe @bikespoke @nplusg @MattJ_O @plainwildcatfan @johndburger @ho,Neutral,21
31440,35239,80191,NaN,06-04-2020,@Jaisans @JamesRider3 @AanthanurDC @doom37455413 @ILuvCO2 @Andrewemcameron @geoffmprice @ElectroPig @glinch72 @IngersolRobert @AgtRigor @Bitakarma @AtomsksSanakan @DevinNunesGoat1 @Article3S @Jamz129 @rm_simmons @RustyAway @JosephsEddy @MikeH_MapleGrov @i,Neutral,21
31657,35456,80408,"North Carolina, USA.",07-04-2020,@ThomONeil1 @bobbiejaneV @media_auntie @BonJS0370 @ldog562 @zazzybritches @pecriel @kristyshl @HurricaneEve81 @MaryHow16 @SafiMaasai @alice4u2010 @ReneTremblay13 @stweetleigh @poppij @Dianne2429 @paulinef136 @mccunecicki @TilghmanChris @StaryStaryNlght @b,Neutral,21


In [10]:
df.drop(df[df['cnt_mentions'] >20].index , inplace=True)

In [11]:
# remove urls 
df['processed'] = df['OriginalTweet'].replace(r'http\S+', ' ', regex=True)
#contractions
df['processed']=df['processed'].apply(contractions.fix)

# convert to lowercase
df['processed']=df['processed'].str.lower()

## removing account names -- basically after @ words -- those are usernames

df['processed']=df['processed'].apply(lambda x :[t for t in x.split() if not t.startswith('@')] ).apply(lambda x: ' '.join(x))
#df['processed'].head(5)

## removing special characters 
df['processed'] = df['processed'].replace(r'[^A-Za-z]+', ' ', regex=True)

##removing extra spaces 
df['processed']=df['processed'].apply(lambda x : x.strip())

## removing stopwords
stop = stopwords.words('english')
stop.extend(['amp','aaa'])

df['processed'] = df['processed'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

print('Shape of dataset',df.shape)

## removing completely blank records 
df=df[df['processed']!='']

print('After removing empty records, shape of the dataset',df.shape)

Shape of dataset (41154, 8)
After removing empty records, shape of the dataset (41122, 8)


In [12]:
## Creating document term matrix using TfidfVectorizer

tfidf = TfidfVectorizer(max_df=0.95, min_df=3, stop_words='english',ngram_range=(1,2))
dtm = tfidf.fit_transform(df['processed'])

 # sparse matrix of 41122(no. of tweets) * 43359(unigrams and bigrams)
dtm       

<41122x43359 sparse matrix of type '<class 'numpy.float64'>'
	with 823194 stored elements in Compressed Sparse Row format>

In [13]:
# creating NMF model with 5 components

nmf_model = NMF(n_components=5,random_state=42)

# fitting and transforming dtm obtained,
# to get weights corresponding to belongingness of the document to each topic

topics=nmf_model.fit_transform(dtm)

In [15]:
# coefficient weights for the document0, 
#higher coefficient weights implies higher chances of belonging into particular topic.

topics[0]  

array([0.        , 0.        , 0.        , 0.04556066, 0.00162339])

In [16]:
len(tfidf.get_feature_names()) ## using get_feature_names property we can have all the unigrams and bigram for which tfidf has been calculated.


43359

In [17]:
#pd.DataFrame(nmf_model.components_,columns=tfidf.get_feature_names()) # 5*43359 matrix for topic and words correspondence.

,aalto,aalto university,aap,aapl,aaron,ab,abandon,abandoned,abating,abbott,abc,abc news,abcnews,abdul,abeg,abeokuta,abhijit,abide,abiding,ability,ability food,able,able access,able afford,able basic,able buy,able continue,able control,able deliver,able demand,able donate,able eat,able food,able grocery,able help,able make,able mask,able meet,able pay,able provide,able purchase,able receive,able send,able shop,able stay,able stock,able supermarket,able visit,able work,ableg,abnormal,abnormal prices,abode,abound,abroad,abrupt,abs,absence,absent,absolute,absolute chaos,absolute panic,absolutely,absolutely brilliant,absolutely disgusted,absolutely disgusting,absolutely essential,absolutely necessary,absolutely need,absolutely wild,absorbed,absurd,abt,abudhabi,abuja,abundance,abundant,abuse,abuse customers,abused,abused supermarket,abuses,abusing,abusive,ac,aca,academic,academy,acc,accc,accelerate,accelerate trend,accelerated,accelerated crisis,accelerates,accelerating,accept,acceptable,acceptance,accepted,accepting,accepting new,access,access capital,access consumer,access coronavirus,access delivery,access essential,access food,access free,access groceries,access health,access healthy,access internet,access online,access ppe,access priority,access soap,access supermarket,access testing,access tests,access vital,accessed,accessibility,accessible,accessing,accessories,accessories giving,accessory,accident,accidental,accidental hoarders,accidental stockpilers,accidentally,accommodate,accommodation,accommodations,accompanied,accomplished,accordance,according,according beijing,according consumer,according data,according experts,according food,according new,according public,according recent,according report,according survey,accordingly,account,account covid,account like,accountability,accountable,accounting,accounts,accounts lose,acct,acct numbers,accuracy,accurate,accurately,accusations,accused,accused price,accused violating,accusing,aceeatery,aceeatery acelounge,acelounge,acesupermarket,acesupermarket aceeatery,achieve,achieved,achieved function,acid,acityunited,acknowledge,acme,acnh,acquire,acquired,acquiring,acquisition,acrylic,act,act covid,act government,act like,act natural,act passed,act quickly,act read,act stop,act vital,acted,acting,acting like,acting responsibly,action,action alert,action covid,action fraud,action govt,action help,action plans,action required,action support,action taken,action today,actions,actions covid,activate,activated,activating,active,active covid,actively,activist,activists,activities,activities like,activity,activity coronavirus,activity covid,activity global,actor,actors,actors famous,actors taking,actress,actress kimberly,acts,acts kindness,actsofkindness,actual,actual footage,actual fuck,actual virus,actually,actually buy,actually covid,actually food,actually going,actually good,actually help,actually need,...,worth coronavirus,worth deliberately,worth food,worth groceries,worth risk,worth rs,worth toilet,worthless,worthy,woul,woven,wow,wow coronavirus,wow good,wow people,wracking,wrap,wrapped,wreaking,wreaking havoc,wreaks,wreaks havoc,wreck,wrecked,wrench,wrenching,wrestlemania,wrestling,write,write ur,writer,writes,writes tesbih,writing,writingcommnunity,writingcommunity,written,wrong,wrong coronavirus,wrong covid,wrong need,wrong people,wrote,wrote letter,wrote thing,wrought,wrt,ws,wsj,wt,wtf,wtf coronavirus,wtf covid,wtf going,wtf wrong,wtfutureipsos,wth,wti,wti prices,wto,wtop,wtrh,wud,wuflu,wuhan,wuhan china,wuhan coronavirus,wuhan grocery,wuhan healthy,wuhan virus,wuhancoronavirus,wuhanvirus,wuhanvirus coronavirus,wv,wv ag,ww,wwd,wwe,wwg,wwg wga,wwii,wwii covid,xauusd,xbox,xd,xmas,xx,ya,ya know,yahoo,yard,yay,ye,yeah,year,year ago,year celebration,year coronavirus,year covid,year end,year food,year high,year low,year lows,year march,year old,year olds,year read,year tops,year year,yearly,yearly resonable,years,years ago,years amid,years consumer,years coronavirus,years covid,years later,

In [29]:
for index,topic in enumerate(nmf_model.components_):
    print(f'THE TOP 10 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

THE TOP 10 WORDS FOR TOPIC #0
['coronavirus covid', 'covid pandemic', 'gas', 'oil prices', 'pandemic', 'coronavirus', 'oil', 'consumer', 'covid', 'prices']


THE TOP 10 WORDS FOR TOPIC #1
['socialdistancing', 'employees', 'going', 'store workers', 'covid', 'coronavirus', 'workers', 'grocery store', 'grocery', 'store']


THE TOP 10 WORDS FOR TOPIC #2
['covid coronavirus', 'coronavirus covid', 'hand sanitizer', 'toiletpaper', 'hand', 'coronavirus', 'sanitizer', 'toilet paper', 'toilet', 'paper']


THE TOP 10 WORDS FOR TOPIC #3
['online grocery', 'coronavirus', 'home', 'grocery shopping', 'delivery', 'shopping online', 'covid', 'online shopping', 'shopping', 'online']


THE TOP 10 WORDS FOR TOPIC #4
['stock', 'coronavirus', 'need', 'covid', 'panic buying', 'buying', 'people', 'panic', 'supermarket', 'food']




In [28]:
df['Topic']=topics.argmax(axis=1)


In [30]:
## Naming topics 
#Topic0-  oil_gas_prices 
#Topic1 - grocery_store_workers
#Topic2- toilet_paper_sanitiser
#Topic3- online_shopping 
#Topic4- panic_buying_hoarding

naming={0:'oil_gas_prices',1:'grocery_store_workers',2:'toilet_paper_sanitiser',3:'online_shopping',4:'panic_buying_hoarding'}
df['Topic_name']=df['Topic'].map(naming)


In [31]:
df[['OriginalTweet','Topic_name']].head()

,OriginalTweet,Topic_name
1,advice Talk to your neighbours family to exchange phone numbers create contact list with phone numbers of neighbours schools employer chemist GP set up online shopping accounts if poss adequate supplies of regular meds but not over order,online_shopping
2,"Coronavirus Australia: Woolworths to give elderly, disabled dedicated shopping hours amid COVID-19 outbreak https://t.co/bInCA9Vp8P",online_shopping
3,"My food stock is not the only one which is empty...\r\r\n\r\r\nPLEASE, don't panic, THERE WILL BE ENOUGH FOOD FOR EVERYONE if you do not take more than you need. \r\r\nStay calm, stay safe.\r\r\n\r\r\n#COVID19france #COVID_19 #COVID19 #coronavirus #confinement #Confinementotal #ConfinementGeneral https://t.co/zrlG0Z520j",panic_buying_hoarding
4,"Me, ready to go at supermarket during the #COVID19 outbreak.\r\r\n\r\r\nNot because I'm paranoid, but because my food stock is litteraly empty. The #coronavirus is a serious thing, but please, don't panic. It causes shortage...\r\r\n\r\r\n#CoronavirusFrance #restezchezvous #StayAtHome #confinement https://t.co/usmuaLq72n",panic_buying_hoarding
5,"As news of the regionÂs first confirmed COVID-19 case came out of Sullivan County last week, people flocked to area stores to purchase cleaning supplies, hand sanitizer, food, toilet paper and other goods, @Tim_Dodson reports https://t.co/cfXch7a2lU",toilet_paper_sanitiser


In [32]:
df['Topic_name'].value_counts()

panic_buying_hoarding     13461
oil_gas_prices            12963
grocery_store_workers      6070
toilet_paper_sanitiser     5078
online_shopping            3550
Name: Topic_name, dtype: int64